In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import plotly
import plotly.graph_objs as go
import warnings
import os
import plotly.io as pio
from sklearn.cluster import KMeans
import numpy as np

warnings.filterwarnings('ignore')

In [2]:
def clear_file(filename='single_factor_analysis.html'):
    with open(filename, mode = "w", encoding="utf-8") as file:
        file.write("")

In [3]:
exec(open('./single_factor_analysis/sfa_code.py').read())

### Read Data & Feature Dictionary

In [4]:
def fill_nas(main, dtypes_df):
    for col in dtypes_df.feature:
        if col in cat_features:
            main[col].fillna("Missing",inplace=True)
        else:
            main[col] = main[col].apply(lambda x: x if (x!=None) and (x!=np.nan) and (np.isnan(x)!=True) else main[col].mean())
    return main

In [ ]:
# main = pd.read_csv("./data/train.csv")
# main.reset_index(inplace=True)
# main.rename(columns={"index":"id","Churn.30":"TARGET"},inplace=True)

In [5]:
sfa_owner_df = pd.read_csv('feature_dictionary.csv')
cont_features = list(sfa_owner_df[sfa_owner_df['dtype2']=='cont']['feature'])
cat_features = list(sfa_owner_df[sfa_owner_df['dtype2']=='cat']['feature'])
all_features = list(sfa_owner_df['feature'])

In [ ]:
# main = fill_nas(main,sfa_owner_df)

### SFA - Continuous Variables

In [6]:
main = pd.read_csv('./data/model_data_jan.csv')
# main.reset_index(inplace=True)
# main.rename(columns={"index":"id","Churn.30":"TARGET"},inplace=True)
# main = fill_nas(main,sfa_owner_df)

In [7]:
main.head()

,id,ultimo_ivr_departamento,ultima_categoria_open_question,ultima_subcategoria_open_question,grupo_acd,total_segundos_espera,total_segundos_conversacion,transferencia,agentes_involucrados,id_agente,...,port_in_operator_ds_clustered_11,client_segment_clustered_2,client_segment_clustered_5,client_segment_clustered_8,client_segment_clustered_11,bpo_clustered_2,total_segundos_espera_transformed,total_segundos_conversacion_transformed,ranking_transformed,tariff_ds_fibra_quota_transformed
0,0,ATC,AVERIAS,MOVIL,60302,1.0,29,False,1,3951,...,3,0,0,0,8,0,0,0,1,0
1,1,ATC,Missing,Missing,60305,642.0,368,False,1,2413,...,0,0,0,7,6,0,1,1,1,0
2,2,ATC,TARIFAS,INFORMACION,69305,83.0,107,False,1,4641,...,5,0,0,7,6,0,1,1,1,0
3,3,ATC,PORTABILIDAD,GENERAL,60302,243.0,155,False,1,2477,...,0,0,0,0,8,0,1,1,0,0
4,4,ATC,INTERNET,ACTIVAR_BONO,60300,1.0,39,False,1,2537,...,5,0,0,0,8,0,0,0,1,0


In [152]:
clear_file("single_factor_analysis_jan.html")

In [8]:
c_values = []
graph_stats_dfs_cont = []
# with open('single_factor_analysis_jan.html', mode = "a", encoding="utf-8") as file:
#     file.write("<b>Continuous Variables Analysis")
    
for feat in cont_features:
    print("="*3+" "+feat+" "+"="*3)
    graph_stats, stats_df, minimum, maximum = sfa_cont(main, feat, 'TARGET', 6,id_column="id")

    exec(open('./single_factor_analysis/draw_table.py').read())
    pio.write_image(fig0, './single_factor_analysis/images/table/'+feat+'_table.png')

    exec(open('./single_factor_analysis/draw_graph.py').read())
    pio.write_image(fig, './single_factor_analysis/images/graph/'+feat+'_graph.png')

    c_values.append([feat,stats_df.iloc[0]['org c-value']])
    graph_stats_dfs_cont.append([feat,graph_stats])

    # file.write(pio.to_html(fig, default_width = '50%', default_height = '50%'))
    # file.write(pio.to_html(fig0))
    print ('OK for '+feat)

c_values_df = pd.DataFrame(c_values, columns=['feature','c_value'])
# c_values_df.sort_values(by='c_value',ascending=False).to_excel('./cont_feature_ranking.xlsx')
c_values_df.sort_values(by='c_value',ascending=False,inplace=True)
# file.write("<b>C-Values Table</b>")
# file.write(c_values_df.to_html())
print('Finished Continuous SFA')
graph_stats_dfs_cont = pd.DataFrame(graph_stats_dfs_cont,columns=['feature','stats_df'])

=== total_segundos_espera ===
Number of not-null observations: 50318
OK for total_segundos_espera
=== total_segundos_conversacion ===
Number of not-null observations: 50318
OK for total_segundos_conversacion
=== agentes_involucrados ===
Number of not-null observations: 50318
OK for agentes_involucrados
=== ranking ===
Number of not-null observations: 50318
OK for ranking
=== client_segment ===
Number of not-null observations: 50318
OK for client_segment
=== bpo ===
Number of not-null observations: 50318
OK for bpo
=== tariff_ds_fibra_quota ===
Number of not-null observations: 50318
OK for tariff_ds_fibra_quota
Finished Continuous SFA


### SFA - Categoric Variables

In [13]:
information_values = []
graph_stats_dfs_cat=[]
# with open('zipcode_sfa.html', mode = "a", encoding="utf-8") as file:
#     file.write("<b>Continuous Variables Analysis")

for feat in cat_features:
#     if feat.find("zip_code")!=-1 or feat.find("locality")!=-1:
        # print("="*3+" "+feat+" "+"="*3)
    graph_stats, stats_df = sfa_cat(main, feat, 'TARGET',id_column="id")

    exec(open('./single_factor_analysis/draw_table_cat.py').read())
    pio.write_image(fig0, './single_factor_analysis/images/table/'+feat+'_table.png')

    exec(open('./single_factor_analysis/draw_graph_cat.py').read())
    pio.write_image(fig, './single_factor_analysis/images/graph/'+feat+'_graph.png')
    information_values.append([feat,stats_df.iloc[0]['iv']])
    graph_stats_dfs_cat.append([feat,graph_stats])

#             file.write(pio.to_html(fig, default_width = '50%', default_height = '50%'))
#             file.write(pio.to_html(fig_cat))
    print ('OK for '+feat)
        
information_values_df = pd.DataFrame(information_values, columns=['feature','iv'])
information_values_df.sort_values(by='iv',ascending=False,inplace=True)
    # information_values_df.sort_values(by='iv',ascending=False).to_excel('./cont_feature_ranking.xlsx')
#     file.write("<b>IV Table</b>")
#     file.write(information_values_df.to_html())
print('Finished Categorical SFA')
graph_stats_dfs_cat = pd.DataFrame(graph_stats_dfs_cat,columns=['feature','stats_df'])

OK for ultimo_ivr_departamento
OK for ultima_categoria_open_question
OK for ultima_subcategoria_open_question
OK for grupo_acd
OK for transferencia
OK for es_rellamada_1h
OK for es_rellamada_24h
OK for es_rellamada_48h
OK for es_rellamada_72h
OK for es_rellamada_agente_1h
OK for es_rellamada_agente_24h
OK for es_rellamada_agente_48h
OK for es_rellamada_agente_72h
OK for billing_type_ds
OK for tariff_ds
OK for service_type
OK for line_type_ds
OK for activation_reason_ds
OK for deactivation_reason_ds
OK for technology_ds
OK for technology_speed_ds
OK for channel_ds
OK for account_bundle_TYPE
OK for bundle_summary
OK for LLAMADA_RETEN_1h
OK for LLAMADA_RETEN_24h
OK for LLAMADA_RETEN_48h
OK for llamada_reten_3D
OK for llamada_reten_4D
OK for llamada_reten_5D
OK for llamada_reten_6D
OK for llamada_reten_7D
OK for times_retention
OK for zip_code
OK for locality_area
OK for province
OK for port_in_operator_ds
OK for outcome_baja
OK for tariff_ds_has_fibra
OK for ultimo_ivr_departamento_cluste

In [140]:
def find_familia(x):
    iy = x.find("transformed")
    ix = x.find("clustered")
    if x.find("transformed")!=-1 or x.find("clustered")!=-1:
        return x[:(max(ix,iy)-1)]
    else:
        return x

In [141]:
information_values_df["familia"] = information_values_df.feature.apply(find_familia)

In [143]:
information_values_df.to_excel("ivs_jan_2.xlsx")

### Apply Clustering (Categorical Features)

In [169]:
def get_clustering_scheme(graph_stats_dfs_cat, steps=2, max_clusters= 10):
    clustering_scheme = {}
    for i in graph_stats_dfs_cat.index:
        tmp = graph_stats_dfs_cat.loc[i].stats_df
        n=tmp.shape[0]
        if n>=max_clusters:
            n=max_clusters
        if n>2:
            clustering_scheme[tmp.index.name] = list(range(2,n,steps))
    return clustering_scheme

In [170]:
def apply_kmeans(graph_stats_dfs_cat, feature, n_clusters=2):
    try:
        tmp = graph_stats_dfs_cat[graph_stats_dfs_cat.feature==feature].iloc[0].stats_df
        X = np.array([[e,f] for e,f in zip([1]*tmp.shape[0],tmp.target_ratio.values)])
        kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
        tmp["category"]=kmeans.labels_
        cluster_map = {}
        for cat in tmp.category.unique():
            cluster_map[cat]=tmp[tmp['category']==cat].index.tolist()
        print(feature+" splitted to "+str(n_clusters)+" categories.")
        return cluster_map
    except:
        print(feature+" can't be splitted to "+str(n_clusters)+" categories.")

In [171]:
def find_cluster(x,cluster_map):
    for key,value in cluster_map.items():
        if(x in value):
            return key
    print("cant find "+str(x)+" in cluster_map")
    return -1

In [172]:
def apply_clusters_to_data(main,graph_stats_dfs_cat,feature,n_clusters=5,suffix="_clustered"):
    cluster_map = apply_kmeans(graph_stats_dfs_cat,feature, n_clusters)
    main[feature+suffix+"_"+str(len(cluster_map.keys()))] = main[feature].apply(find_cluster,args=(cluster_map,))
    return main

In [173]:
clustering_scheme = get_clustering_scheme(graph_stats_dfs_cat,steps=3,max_clusters=12)

In [174]:
clustering_scheme

{'zip_code': [2, 5, 8, 11],
 'locality_area': [2, 5, 8, 11],
 'zip_code_clustered_5': [2],
 'zip_code_clustered_8': [2, 5],
 'zip_code_clustered_11': [2, 5, 8],
 'locality_area_clustered_5': [2],
 'locality_area_clustered_8': [2, 5],
 'locality_area_clustered_11': [2, 5, 8]}

In [130]:
for feature in clustering_scheme.keys():
    for i in clustering_scheme[feature]:
        print(feature + " - "+ str(i))
        main = apply_clusters_to_data(main, graph_stats_dfs_cat, feature, n_clusters=i)

ultimo_ivr_departamento - 2
ultimo_ivr_departamento splitted to 2 categories.
ultimo_ivr_departamento - 5
ultimo_ivr_departamento splitted to 5 categories.
ultima_categoria_open_question - 2
ultima_categoria_open_question splitted to 2 categories.
ultima_categoria_open_question - 5
ultima_categoria_open_question splitted to 5 categories.
ultima_categoria_open_question - 8
ultima_categoria_open_question splitted to 8 categories.
ultima_categoria_open_question - 11
ultima_categoria_open_question splitted to 11 categories.
ultima_subcategoria_open_question - 2
ultima_subcategoria_open_question splitted to 2 categories.
ultima_subcategoria_open_question - 5
ultima_subcategoria_open_question splitted to 5 categories.
ultima_subcategoria_open_question - 8
ultima_subcategoria_open_question splitted to 8 categories.
ultima_subcategoria_open_question - 11
ultima_subcategoria_open_question splitted to 11 categories.
grupo_acd - 2
grupo_acd splitted to 2 categories.
grupo_acd - 5
grupo_acd splitt

### Apply Threshold (Continuous Features)

In [131]:
main["total_segundos_espera_transformed"] = (main["total_segundos_espera"]>25)*1
main["total_segundos_conversacion_transformed"] = (main["total_segundos_conversacion"]>75)*1
main["ranking_transformed"] = (main["ranking"]==1)*1
main["tariff_ds_fibra_quota_transformed"] = (main["tariff_ds_fibra_quota"]==300)*1

### Export Final Data

In [132]:
main.to_csv('./data/model_data_jan.csv',index=False)